<a href="https://colab.research.google.com/github/ez-zee/Annotation-Automation-Scripts-Python-/blob/main/roboflow_annotated_dataset_to_cvat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import json
import requests
from glob import glob
from cv2 import imread
from tqdm import tqdm
from cvat_sdk.api_client import Configuration, ApiClient
from cvat_sdk.api_client.api.tasks_api import TasksApi

# === Конфигурация ===
IMAGE_DIR = r"C:\extracted_videoframes"
COCO_ANNOTATION_PATH = os.path.join(IMAGE_DIR, "coco_keypoints_export.json")

# Roboflow
ROBOFLOW_API_KEY = "Dpg4wVjxAMcUGN4QlzRY"
ROBOFLOW_MODEL = "football-players-dvgnm/1"
ROBOFLOW_ENDPOINT = f"https://detect.roboflow.com/{ROBOFLOW_MODEL}?api_key={ROBOFLOW_API_KEY}"

# CVAT
CVAT_HOST = "http://localhost:8080"
CVAT_USERNAME = "Azat"
CVAT_PASSWORD = "********"
TASK_ID = 4

# === Шаг 1: Инференс через Roboflow ===
def infer_with_roboflow():
    images = glob(os.path.join(IMAGE_DIR, "*.jpg"))
    annotations = []
    coco_images = []

    categories = [{
        "id": 1,
        "name": "person",
        "supercategory": "person",
        "keypoints": [
            "right-shoulder", "left-shoulder", "right-elbow", "left-elbow",
            "right-wrist", "left-wrist", "right-hip", "left-hip",
            "right-knee", "left-knee", "right-ankle", "left-ankle"
        ],
        "skeleton": [
            [1, 3], [2, 4], [3, 5], [4, 6], [7, 9], [8, 10], [9, 11], [10, 12]
        ]
    }]

    ann_id = 1
    print(f"[INFO] Обработка {len(images)} изображений...")

    for img_id, image_path in enumerate(tqdm(images), 1):
        with open(image_path, "rb") as img_file:
            response = requests.post(
                ROBOFLOW_ENDPOINT,
                files={"file": img_file},
                data={"confidence": 10, "overlap": 30}
            )

        if response.status_code != 200:
            print(f"[ОШИБКА] {os.path.basename(image_path)}: {response.status_code}")
            continue

        result = response.json()
        height, width = imread(image_path).shape[:2]

        coco_images.append({
            "id": img_id,
            "file_name": os.path.basename(image_path),
            "width": width,
            "height": height
        })

        for pred in result.get("predictions", []):
            keypoints = []
            num_keypoints = 0

            for kp in pred.get("keypoints", []):
                keypoints.extend([kp["x"], kp["y"], 2])  # visibility=2
                num_keypoints += 1

            annotations.append({
                "id": ann_id,
                "image_id": img_id,
                "category_id": 1,
                "keypoints": keypoints,
                "num_keypoints": num_keypoints,
                "bbox": [pred["x"], pred["y"], pred["width"], pred["height"]],
                "iscrowd": 0,
                "area": pred["width"] * pred["height"]
            })

            ann_id += 1

    coco_data = {
        "info": {"description": "Auto-annotated dataset from Roboflow"},
        "licenses": [],
        "images": coco_images,
        "annotations": annotations,
        "categories": categories
    }

    with open(COCO_ANNOTATION_PATH, "w", encoding="utf-8") as f:
        json.dump(coco_data, f, indent=2)

    print(f"✅ COCO JSON сохранён: {COCO_ANNOTATION_PATH}")

# === Шаг 2: Импорт COCO-аннотаций в CVAT ===
def import_to_cvat():
    print("[INFO] Подключаемся к CVAT...")
    configuration = Configuration(
        host=CVAT_HOST,
        username=CVAT_USERNAME,
        password=CVAT_PASSWORD
    )

    with ApiClient(configuration) as api_client:
        tasks_api = TasksApi(api_client)
        print("[INFO] Загружаем COCO-аннотации в задание...")

        with open(COCO_ANNOTATION_PATH, "rb") as file_data:
            from cvat_sdk.api_client.models import AnnotationFileRequest
            request = AnnotationFileRequest(
                annotation_file=file_data,
                annotation_format="COCO Keypoints 1.0",
                filename="coco_keypoints_export.json"
            )
            tasks_api.update_annotations(id=TASK_ID, data=request)


        print("✅ Аннотации успешно загружены в CVAT")

# === Запуск ===
if __name__ == "__main__":
    infer_with_roboflow()
    import_to_cvat()
